---
## 1. Carga de Datos

---

## 1. Configuración e Importación de Librerías

In [1]:
import pandas as pd
import json
import os
from datetime import datetime
from IPython.display import display, Markdown, JSON

# Configuración de pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

---

## 2. Carga de la Tabla de Hechos

Cargamos los datos exportados de la base de datos del Data Warehouse. Esta tabla contiene todos los indicadores económicos procesados durante el proyecto ETL.

In [2]:
# Ruta al archivo exportado de la base de datos
db_export_path = "../datos/procesados/hechos_exportados_db.csv"

if os.path.exists(db_export_path):
    df_hechos = pd.read_csv(db_export_path)
    df_hechos['fecha'] = pd.to_datetime(df_hechos['fecha'])
    display(Markdown(f"**Archivo cargado correctamente:** `{db_export_path}`"))
else:
    raise FileNotFoundError(f"No se encontró el archivo: {db_export_path}")

**Archivo cargado correctamente:** `../datos/procesados/hechos_exportados_db.csv`

### 2.1 Resumen de la Tabla de Hechos

In [3]:
# Información general del dataset
resumen = pd.DataFrame({
    'Métrica': [
        'Total de registros',
        'Columnas',
        'Fecha mínima',
        'Fecha máxima',
        'Indicadores únicos',
        'Geografías'
    ],
    'Valor': [
        f"{len(df_hechos):,}",
        ', '.join(df_hechos.columns.tolist()),
        df_hechos['fecha'].min().strftime('%Y-%m-%d'),
        df_hechos['fecha'].max().strftime('%Y-%m-%d'),
        df_hechos['indicador'].nunique(),
        ', '.join(df_hechos['geografia'].unique())
    ]
})
resumen.style.set_properties(**{'text-align': 'left'}).hide(axis='index')

Métrica,Valor
Total de registros,"16,526"
Columnas,"fecha, indicador, indicador_codigo, valor, unidad, geografia, fuente"
Fecha mínima,2010-01-01
Fecha máxima,2025-12-31
Indicadores únicos,31
Geografías,"Global, España"


### 2.2 Muestra de los Datos

In [4]:
df_hechos.head(10)

,fecha,indicador,indicador_codigo,valor,unidad,geografia,fuente
0,2010-01-01,Tipo Cambio EUR/USD,EURUSD,1.4390,USD/EUR,Global,Yahoo Finance
1,2010-01-04,Tipo Cambio EUR/USD,EURUSD,1.4424,USD/EUR,Global,Yahoo Finance
2,2010-01-04,Precio Oro EUR,XAU_EUR,774.8902,EUR/oz,Global,Yahoo Finance
3,2010-01-04,IBEX 35,IBEX35,12145.0869,Puntos,España,Yahoo Finance
4,2010-01-04,Precio Oro USD,XAU_USD,1117.7000,USD/oz,Global,Yahoo Finance
5,2010-01-05,Precio Oro USD,XAU_USD,1118.1000,USD/oz,Global,Yahoo Finance
6,2010-01-05,IBEX 35,IBEX35,12204.3877,Puntos,España,Yahoo Finance
7,2010-01-05,Tipo Cambio EUR/USD,EURUSD,1.4366,USD/EUR,Global,Yahoo Finance
8,2010-01-05,Precio Oro EUR,XAU_EUR,778.2982,EUR/oz,Global,Yahoo Finance
9,2010-01-06,Tipo Cambio EUR/USD,EURUSD,1.4404,USD/EUR,Global,Yahoo Finance


---

## 3. Exploración de Indicadores Disponibles

Analizamos los diferentes indicadores económicos que serán transformados a Schema.org.

In [5]:
# Conteo de registros por indicador
indicadores_resumen = df_hechos.groupby('indicador').agg(
    registros=('valor', 'count'),
    valor_min=('valor', 'min'),
    valor_max=('valor', 'max'),
    unidad=('unidad', 'first'),
    fuente=('fuente', 'first')
).reset_index()

indicadores_resumen = indicadores_resumen.sort_values('registros', ascending=False)
indicadores_resumen

,indicador,registros,valor_min,valor_max,unidad,fuente
30,Tipo Cambio EUR/USD,4122,0.9596,1.4844,USD/EUR,Yahoo Finance
13,IBEX 35,4049,5956.2940,16150.0996,Puntos,Yahoo Finance
16,Precio Oro USD,3981,1050.8001,4336.3999,USD/oz,Yahoo Finance
15,Precio Oro EUR,3975,769.1476,3719.8505,EUR/oz,Yahoo Finance
5,Desempleo 35-39,16,8.8900,13.9200,%,INE
...,...,...,...,...,...,...
24,Salario Decil 70-80,14,2291.7000,2782.0000,EUR,INE
29,Salario medio bruto,14,22594.8700,27981.9000,EUR,INE
28,Salario mediano,13,18871.5400,22141.5900,EUR,INE
27,Salario mas frecuente,13,14522.7900,17031.9000,EUR,INE


---

## 4. Definición del Esquema de Mapeo a Schema.org

### 4.1 Tipo de Entidad: `Observation`

Utilizamos el tipo **`Observation`** de Schema.org, que es ideal para representar mediciones y observaciones de datos estadísticos.

| Campo Schema.org | Descripción | Mapeo desde nuestra BD |
|------------------|-------------|------------------------|
| `@type` | Tipo de entidad | "Observation" (fijo) |
| `@id` | Identificador único | Generado: indicador + fecha |
| `observationDate` | Fecha de la observación | `fecha` |
| `measuredValue` | Valor numérico medido | `valor` |
| `unitText` | Unidad de medida | `unidad` |
| `variableMeasured` | Variable que se mide | `indicador` |
| `spatialCoverage` | Cobertura geográfica | `geografia` |
| `sdPublisher` | Fuente de los datos | `fuente` |

In [6]:
# Definición de la función de transformación
def crear_observation_jsonld(row):
    """
    Transforma una fila del DataFrame de hechos en un diccionario JSON-LD 
    siguiendo el esquema Observation de Schema.org.
    
    Args:
        row: Fila del DataFrame con los campos fecha, indicador, valor, unidad, geografia, fuente
    
    Returns:
        dict: Objeto JSON-LD válido según Schema.org
    """
    # Crear un identificador único seguro (sin espacios ni caracteres especiales)
    indicador_slug = str(row['indicador']).replace(' ', '_').replace('/', '_')
    fecha_str = row['fecha'].strftime('%Y-%m-%d')
    obs_id = f"https://example.org/fiat-depreciacion/observation/{indicador_slug}/{fecha_str}"
    
    # Determinar cobertura geográfica
    geografia = row.get('geografia', 'Global')
    
    # Construir el objeto JSON-LD
    observation = {
        "@context": "https://schema.org",
        "@type": "Observation",
        "@id": obs_id,
        "observationDate": fecha_str,
        "measuredValue": float(row['valor']),
        "unitText": str(row['unidad']),
        "variableMeasured": {
            "@type": "PropertyValue",
            "name": str(row['indicador']),
            "propertyID": str(row.get('indicador_codigo', row['indicador']))
        },
        "spatialCoverage": {
            "@type": "Place",
            "name": geografia
        },
        "sdPublisher": {
            "@type": "Organization",
            "name": str(row.get('fuente', 'Unknown'))
        }
    }
    
    return observation

### 4.2 Ejemplo de Transformación

Veamos cómo se transforma un registro individual:

In [7]:
# Ejemplo con el primer registro
ejemplo_original = df_hechos.iloc[0]
display(Markdown("**Registro Original:**"))
display(pd.DataFrame([ejemplo_original]))

**Registro Original:**

,fecha,indicador,indicador_codigo,valor,unidad,geografia,fuente
0,2010-01-01,Tipo Cambio EUR/USD,EURUSD,1.439,USD/EUR,Global,Yahoo Finance


In [8]:
# Transformación a JSON-LD
ejemplo_jsonld = crear_observation_jsonld(ejemplo_original)
display(Markdown("**Resultado JSON-LD:**"))
print(json.dumps(ejemplo_jsonld, indent=2, ensure_ascii=False))

**Resultado JSON-LD:**

{
  "@context": "https://schema.org",
  "@type": "Observation",
  "@id": "https://example.org/fiat-depreciacion/observation/Tipo_Cambio_EUR_USD/2010-01-01",
  "observationDate": "2010-01-01",
  "measuredValue": 1.439,
  "unitText": "USD/EUR",
  "variableMeasured": {
    "@type": "PropertyValue",
    "name": "Tipo Cambio EUR/USD",
    "propertyID": "EURUSD"
  },
  "spatialCoverage": {
    "@type": "Place",
    "name": "Global"
  },
  "sdPublisher": {
    "@type": "Organization",
    "name": "Yahoo Finance"
  }
}


---

## 5. Transformación Completa de la Tabla de Hechos

Aplicamos la transformación a **todos los registros** de la tabla de hechos para generar el archivo JSON-LD completo.

In [9]:
# Transformar todos los registros
json_ld_observations = []

for idx, row in df_hechos.iterrows():
    observation = crear_observation_jsonld(row)
    json_ld_observations.append(observation)

# Resumen de la transformación
transformacion_resumen = pd.DataFrame({
    'Metrica': [
        'Registros procesados',
        'Observaciones JSON-LD generadas',
        'Tasa de conversión'
    ],
    'Valor': [
        f"{len(df_hechos):,}",
        f"{len(json_ld_observations):,}",
        f"{(len(json_ld_observations)/len(df_hechos))*100:.1f}%"
    ]
})
transformacion_resumen.style.set_properties(**{'text-align': 'left'}).hide(axis='index')

Metrica,Valor
Registros procesados,"16,526"
Observaciones JSON-LD generadas,"16,526"
Tasa de conversión,100.0%


### 5.1 Estructura del Documento JSON-LD Final

El documento final se estructura como un `@graph` que contiene todas las observaciones, junto con metadatos del dataset.

In [10]:
# Crear documento JSON-LD completo con metadatos
documento_jsonld = {
    "@context": "https://schema.org",
    "@type": "Dataset",
    "@id": "https://example.org/fiat-depreciacion/dataset",
    "name": "Depreciacion del Euro - Indicadores Economicos (2010-2025)",
    "description": "Dataset que contiene indicadores economicos para analizar la depreciacion del Euro frente a activos reales como el Oro y la Bolsa (IBEX35), asi como su impacto en los salarios espanoles.",
    "temporalCoverage": f"{df_hechos['fecha'].min().strftime('%Y-%m-%d')}/{df_hechos['fecha'].max().strftime('%Y-%m-%d')}",
    "spatialCoverage": {
        "@type": "Place",
        "name": "Spain, Global"
    },
    "creator": {
        "@type": "Organization",
        "name": "Universidad - Proyecto Integracion de Datos"
    },
    "dateCreated": datetime.now().strftime('%Y-%m-%d'),
    "license": "https://creativecommons.org/licenses/by/4.0/",
    "variableMeasured": [
        {"@type": "PropertyValue", "name": ind} 
        for ind in df_hechos['indicador'].unique()
    ],
    "distribution": {
        "@type": "DataDownload",
        "encodingFormat": "application/ld+json",
        "contentUrl": "https://example.org/fiat-depreciacion/output_schema.jsonld"
    },
    "hasPart": json_ld_observations
}

display(Markdown(f"**Documento JSON-LD creado con {len(json_ld_observations):,} observaciones**"))

**Documento JSON-LD creado con 16,526 observaciones**

### 5.2 Vista Previa del Documento (Primeras 3 Observaciones)

In [11]:
# Mostrar estructura con solo las primeras observaciones
preview = documento_jsonld.copy()
preview['hasPart'] = preview['hasPart'][:3]
preview['hasPart'].append({"...": f"... y {len(json_ld_observations)-3:,} observaciones mas"})

print(json.dumps(preview, indent=2, ensure_ascii=False))

{
  "@context": "https://schema.org",
  "@type": "Dataset",
  "@id": "https://example.org/fiat-depreciacion/dataset",
  "name": "Depreciacion del Euro - Indicadores Economicos (2010-2025)",
  "description": "Dataset que contiene indicadores economicos para analizar la depreciacion del Euro frente a activos reales como el Oro y la Bolsa (IBEX35), asi como su impacto en los salarios espanoles.",
  "temporalCoverage": "2010-01-01/2025-12-31",
  "spatialCoverage": {
    "@type": "Place",
    "name": "Spain, Global"
  },
  "creator": {
    "@type": "Organization",
    "name": "Universidad - Proyecto Integracion de Datos"
  },
  "dateCreated": "2025-12-02",
  "license": "https://creativecommons.org/licenses/by/4.0/",
  "variableMeasured": [
    {
      "@type": "PropertyValue",
      "name": "Tipo Cambio EUR/USD"
    },
    {
      "@type": "PropertyValue",
      "name": "Precio Oro EUR"
    },
    {
      "@type": "PropertyValue",
      "name": "IBEX 35"
    },
    {
      "@type": "Propert

---

## 6. Exportación del Archivo JSON-LD

Guardamos el documento completo en el archivo `output_schema.jsonld`.

In [12]:
# Ruta de salida
output_file = "output_schema.jsonld"

# Guardar archivo
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(documento_jsonld, f, indent=2, ensure_ascii=False)

# Verificar tamaño del archivo
file_size = os.path.getsize(output_file)

# Resumen de exportación
export_resumen = pd.DataFrame({
    'Propiedad': [
        'Archivo generado',
        'Tamano del archivo',
        'Total de observaciones',
        'Indicadores incluidos',
        'Rango temporal',
        'Formato'
    ],
    'Valor': [
        output_file,
        f"{file_size / 1024 / 1024:.2f} MB" if file_size > 1024*1024 else f"{file_size / 1024:.1f} KB",
        f"{len(json_ld_observations):,}",
        f"{df_hechos['indicador'].nunique()}",
        f"{df_hechos['fecha'].min().strftime('%Y')} - {df_hechos['fecha'].max().strftime('%Y')}",
        'JSON-LD (Schema.org)'
    ]
})
export_resumen.style.set_properties(**{'text-align': 'left'}).hide(axis='index')

Propiedad,Valor
Archivo generado,output_schema.jsonld
Tamano del archivo,9.69 MB
Total de observaciones,"16,526"
Indicadores incluidos,31
Rango temporal,2010 - 2025
Formato,JSON-LD (Schema.org)


---

## 7. Validación del JSON-LD Generado

Verificamos que el archivo generado es válido y contiene la estructura esperada.

In [13]:
# Leer y validar el archivo generado
with open(output_file, 'r', encoding='utf-8') as f:
    documento_leido = json.load(f)

# Validaciones
validaciones = [
    ('Contexto Schema.org presente', documento_leido.get('@context') == 'https://schema.org'),
    ('Tipo Dataset correcto', documento_leido.get('@type') == 'Dataset'),
    ('ID unico presente', '@id' in documento_leido),
    ('Nombre del dataset', 'name' in documento_leido),
    ('Cobertura temporal', 'temporalCoverage' in documento_leido),
    ('Observaciones incluidas', 'hasPart' in documento_leido and len(documento_leido['hasPart']) > 0),
    ('Cantidad correcta', len(documento_leido.get('hasPart', [])) == len(json_ld_observations))
]

validacion_df = pd.DataFrame(validaciones, columns=['Validacion', 'Resultado'])
validacion_df['Estado'] = validacion_df['Resultado'].apply(lambda x: 'OK' if x else 'ERROR')
validacion_df[['Validacion', 'Estado']]

,Validacion,Estado
0,Contexto Schema.org presente,OK
1,Tipo Dataset correcto,OK
2,ID unico presente,OK
3,Nombre del dataset,OK
4,Cobertura temporal,OK
5,Observaciones incluidas,OK
6,Cantidad correcta,OK


### 7.1 Muestra de Observaciones por Indicador

In [14]:
# Mostrar una observación de ejemplo por cada indicador
observaciones_muestra = []
indicadores_vistos = set()

for obs in documento_leido['hasPart']:
    indicador = obs['variableMeasured']['name']
    if indicador not in indicadores_vistos:
        indicadores_vistos.add(indicador)
        observaciones_muestra.append({
            'Indicador': indicador,
            'Fecha': obs['observationDate'],
            'Valor': obs['measuredValue'],
            'Unidad': obs['unitText'],
            'Geografia': obs['spatialCoverage']['name']
        })

pd.DataFrame(observaciones_muestra)

,Indicador,Fecha,Valor,Unidad,Geografia
0,Tipo Cambio EUR/USD,2010-01-01,1.439000e+00,USD/EUR,Global
1,Precio Oro EUR,2010-01-04,7.748902e+02,EUR/oz,Global
2,IBEX 35,2010-01-04,1.214509e+04,Puntos,España
3,Precio Oro USD,2010-01-04,1.117700e+03,USD/oz,Global
4,Salario Decil 80-90,2010-12-31,2.841700e+03,EUR,España
...,...,...,...,...,...
26,Salario Decil 30-40,2010-12-31,1.316050e+03,EUR,España
27,Salario mas frecuente,2010-12-31,1.452279e+04,EUR,España
28,Salario mediano,2010-12-31,1.887154e+04,EUR,España
29,Salario medio bruto,2010-12-31,2.260748e+04,EUR,España


---

## 8. Conclusiones

### Resumen del Proceso

Se ha completado exitosamente la transformación de la tabla de hechos del proyecto de depreciación del Euro al formato JSON-LD siguiendo el vocabulario de Schema.org.

### Beneficios de esta Transformación

| Beneficio | Descripción |
|-----------|-------------|
| **Interoperabilidad** | Los datos pueden ser consumidos por cualquier sistema que entienda Schema.org |
| **SEO** | Los motores de búsqueda pueden indexar y entender los datos estructurados |
| **Linked Data** | Los datos están preparados para integrarse en la web semántica |
| **Estandarización** | Se utiliza un vocabulario ampliamente adoptado y mantenido |
| **Documentación** | El esquema es autodescriptivo y fácil de entender |

### Archivo Generado

El archivo `output_schema.jsonld` contiene todas las observaciones del proyecto y puede ser utilizado para:

- Publicación en la web como datos estructurados
- Integración con APIs de datos abiertos
- Importación en sistemas de gestión de conocimiento
- Validación con herramientas como el [Schema Markup Validator](https://validator.schema.org/) de Google